In [1]:
import pandas as pd

In [2]:
tweets_hombres = pd.read_csv("/notebooks/notebooks/tweets_hombres.csv",index_col = "Unnamed: 0")
tweets_mujeres = pd.read_csv("/notebooks/notebooks/tweets_mujeres.csv",index_col = "Unnamed: 0")

In [3]:
hombres = list(tweets_hombres["tweet"])
mujeres = list(tweets_mujeres["tweet"])

In [6]:
def add_recurrences(palabras):
    global diccionario
    for word in palabras:
        if "https:" in word: continue
        if word in diccionario.keys():
            diccionario[word] += 1
        else:
            diccionario[word] = 1

In [7]:
def process(corpus):
    global diccionario
    
    count = 0
    diccionario = {}
    for enunciado in corpus:
        
        lista_palabras = enunciado.split()
        add_recurrences(lista_palabras)
        count += 1
    return diccionario

In [8]:
dic_mujeres = process(mujeres)

In [9]:
dic_hombres = process(hombres)

In [10]:
len(dic_mujeres), len(dic_hombres)

(228100, 141364)

In [23]:
df_m = pd.DataFrame({"word":list(dic_mujeres.keys()), "frequence":list(dic_mujeres.values())})
df_h = pd.DataFrame({"word":list(dic_hombres.keys()), "frequence":list(dic_hombres.values())})

In [24]:
df_m.set_index("word",inplace=True)
df_h.set_index("word",inplace=True)

In [27]:
df_m = df_m.sort_values(by=["frequence"],ascending=False)
df_h = df_h.sort_values(by=["frequence"],ascending=False)

# Encoding

In [31]:
from transformers import BertTokenizer, BertConfig
from transformers import BertForPreTraining, BertForMaskedLM

In [36]:
mujeres_model_filepath = '/notebooks/notebooks/my_model_checkpoints_ellas/' 
hombres_model_filepath = '/notebooks/notebooks/my_model_checkpoints_ellos/'

mujeres_model = BertTokenizer.from_pretrained(mujeres_model_filepath)
hombres_model = BertTokenizer.from_pretrained(hombres_model_filepath)

In [45]:
def encode_corpus(model, diccionario):
    embeddings = {}
    for i in diccionario.keys():
        embeddings[i] = model.encode(i, return_tensors="np",padding="max_length",max_length=5,truncation=True)
    return embeddings

In [46]:
embeddings_mujeres = encode_corpus(mujeres_model, dic_mujeres)

In [47]:
embeddings_hombres = encode_corpus(hombres_model, dic_hombres)

In [48]:
df_m = pd.DataFrame({"word":list(dic_mujeres.keys()), 
                     "frequence":list(dic_mujeres.values()),
                     "embedding":list(embeddings_mujeres.values())})

df_h = pd.DataFrame({"word":list(dic_hombres.keys()), 
                     "frequence":list(dic_hombres.values()),
                    "embedding":list(embeddings_hombres.values())})

In [49]:
df_m.set_index("word",inplace=True)
df_h.set_index("word",inplace=True)

df_m = df_m.sort_values(by=["frequence"],ascending=False)
df_h = df_h.sort_values(by=["frequence"],ascending=False)

In [51]:
df_m.to_csv("embeddings/mujeres_encoded.csv")
df_h.to_csv("embeddings/hombres_encoded.csv")